In [8]:
# Enhanced ConvNeXt Seismic Inversion with L1+SSIM Loss
# This notebook improves upon the baseline with better loss function and training

# Configuration
RUN_TRAIN = True
RUN_VALID = True  
RUN_TEST = True

import torch
if not torch.cuda.is_available() or torch.cuda.device_count() < 2:
    raise RuntimeError("Requires >= 2 GPUs with CUDA enabled.")

# Install dependencies
try: 
    import monai
except: 
    !pip install --no-deps monai -q

try:
    import piq
except:
    !pip install piq -q

print("🚀 Enhanced ConvNeXt Training with L1+SSIM Loss")
print("=" * 50)

<frozen importlib._bootstrap_external>:1241: FutureWarning: The cuda.cudart module is deprecated and will be removed in a future release, please switch to use the cuda.bindings.runtime module instead.
2025-05-23 03:40:19.564554: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1747971619.762520      79 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1747971619.822010      79 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


🚀 Enhanced ConvNeXt Training with L1+SSIM Loss


In [9]:
%%writefile _cfg.py

from types import SimpleNamespace
import torch

cfg = SimpleNamespace()
cfg.device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
cfg.local_rank = 0
cfg.seed = 123
cfg.subsample = None

cfg.backbone = "convnext_small.fb_in22k_ft_in1k"
cfg.ema = True
cfg.ema_decay = 0.999

# Enhanced training parameters
cfg.epochs = 50
cfg.batch_size = 16
cfg.batch_size_val = 16

cfg.early_stopping = {"patience": 7, "streak": 0}
cfg.logging_steps = 50

Overwriting _cfg.py


In [10]:
%%writefile _dataset.py

import os
import glob

import numpy as np
import pandas as pd
from tqdm import tqdm

import torch

class CustomDataset(torch.utils.data.Dataset):
    def __init__(
        self, 
        cfg,
        mode = "train", 
    ):
        self.cfg = cfg
        self.mode = mode
        
        self.data, self.labels, self.records = self.load_metadata()

    def load_metadata(self, ):

        # Select rows
        df= pd.read_csv("/kaggle/input/openfwi-preprocessed-72x72/folds.csv")
        if self.cfg.subsample is not None:
            df= df.groupby(["dataset", "fold"]).head(self.cfg.subsample)

        if self.mode == "train":
            df= df[df["fold"] != 0]
        else:
            df= df[df["fold"] == 0]

        
        data = []
        labels = []
        records = []
        mmap_mode = "r"

        for idx, row in tqdm(df.iterrows(), total=len(df), disable=self.cfg.local_rank != 0):
            row= row.to_dict()

            # Hacky way to get exact file name
            p1 = os.path.join("/kaggle/input/open-wfi-1/openfwi_float16_1/", row["data_fpath"])
            p2 = os.path.join("/kaggle/input/open-wfi-1/openfwi_float16_1/", row["data_fpath"].split("/")[0], "*", row["data_fpath"].split("/")[-1])
            p3 = os.path.join("/kaggle/input/open-wfi-2/openfwi_float16_2/", row["data_fpath"])
            p4 = os.path.join("/kaggle/input/open-wfi-2/openfwi_float16_2/", row["data_fpath"].split("/")[0], "*", row["data_fpath"].split("/")[-1])
            farr= glob.glob(p1) + glob.glob(p2) + glob.glob(p3) + glob.glob(p4)
        
            # Map to lbl fpath
            farr= farr[0]
            flbl= farr.replace('seis', 'vel').replace('data', 'model')
            
            # Load
            arr= np.load(farr, mmap_mode=mmap_mode)
            lbl= np.load(flbl, mmap_mode=mmap_mode)

            # Append
            data.append(arr)
            labels.append(lbl)
            records.append(row["dataset"])

        return data, labels, records

    def __getitem__(self, idx):
        row_idx= idx // 500
        col_idx= idx % 500

        d= self.records[row_idx]
        x= self.data[row_idx][col_idx, ...]
        y= self.labels[row_idx][col_idx, ...]

        # Augs 
        if self.mode == "train":
            
            # Temporal flip
            if np.random.random() < 0.5:
                x= x[::-1, :, ::-1]
                y= y[..., ::-1]

        x= x.copy()
        y= y.copy()
        
        return x, y

    def __len__(self, ):
        return len(self.records) * 500

Overwriting _dataset.py


In [11]:
%%writefile _model.py

from copy import deepcopy
from types import MethodType

import torch
import torch.nn as nn
import torch.nn.functional as F

import timm
from timm.models.convnext import ConvNeXtBlock

from monai.networks.blocks import UpSample, SubpixelUpsample
from piq import ssim

####################
## Enhanced Loss  ##
####################

class L1_SSIM_Loss(nn.Module):
    def __init__(self, alpha=0.85):
        super().__init__()
        self.alpha = alpha
        self.l1 = nn.L1Loss()

    def forward(self, pred, target):
        # Compute L1 in whatever dtype we're using
        l1 = self.l1(pred, target)

        # Switch to float32 for SSIM
        pred = pred.float()
        target = target.float()

        min_val = target.amin()
        max_val = target.amax()
        eps = 1e-6

        pred_norm = (pred - min_val) / (max_val - min_val + eps)
        target_norm = (target - min_val) / (max_val - min_val + eps)

        pred_norm = pred_norm.clamp(0.0, 1.0)
        target_norm = target_norm.clamp(0.0, 1.0)

        ssim_val = ssim(pred_norm, target_norm, data_range=1.0)

        return self.alpha * l1 + (1.0 - self.alpha) * (1.0 - ssim_val)

####################
## EMA + Ensemble ##
####################

class ModelEMA(nn.Module):
    def __init__(self, model, decay=0.99, device=None):
        super().__init__()
        self.module = deepcopy(model)
        self.module.eval()
        self.decay = decay
        self.device = device
        if self.device is not None:
            self.module.to(device=device)

    def _update(self, model, update_fn):
        with torch.no_grad():
            for ema_v, model_v in zip(self.module.state_dict().values(), model.state_dict().values()):
                if self.device is not None:
                    model_v = model_v.to(device=self.device)
                ema_v.copy_(update_fn(ema_v, model_v))

    def update(self, model):
        self._update(model, update_fn=lambda e, m: self.decay * e + (1. - self.decay) * m)

    def set(self, model):
        self._update(model, update_fn=lambda e, m: m)


class EnsembleModel(nn.Module):
    def __init__(self, models):
        super().__init__()
        self.models = nn.ModuleList(models).eval()

    def forward(self, x):
        output = None
        
        for m in self.models:
            logits= m(x)
            
            if output is None:
                output = logits
            else:
                output += logits
                
        output /= len(self.models)
        return output

#############
## Decoder ##
#############

class ConvBnAct2d(nn.Module):
    def __init__(
        self,
        in_channels,
        out_channels,
        kernel_size,
        padding: int = 0,
        stride: int = 1,
        norm_layer: nn.Module = nn.Identity,
        act_layer: nn.Module = nn.ReLU,
    ):
        super().__init__()

        self.conv= nn.Conv2d(
            in_channels, 
            out_channels,
            kernel_size,
            stride=stride, 
            padding=padding, 
            bias=False,
        )
        self.norm = norm_layer(out_channels) if norm_layer != nn.Identity else nn.Identity()
        self.act= act_layer(inplace=True)

    def forward(self, x):
        x = self.conv(x)
        x = self.norm(x)
        x = self.act(x)
        return x


class SCSEModule2d(nn.Module):
    def __init__(self, in_channels, reduction=16):
        super().__init__()
        self.cSE = nn.Sequential(
            nn.AdaptiveAvgPool2d(1),
            nn.Conv2d(in_channels, in_channels // reduction, 1),
            nn.Tanh(),
            nn.Conv2d(in_channels // reduction, in_channels, 1),
            nn.Sigmoid(),
        )
        self.sSE = nn.Sequential(
            nn.Conv2d(in_channels, 1, 1), 
            nn.Sigmoid(),
            )

    def forward(self, x):
        return x * self.cSE(x) + x * self.sSE(x)

class Attention2d(nn.Module):
    def __init__(self, name, **params):
        super().__init__()
        if name is None:
            self.attention = nn.Identity(**params)
        elif name == "scse":
            self.attention = SCSEModule2d(**params)
        else:
            raise ValueError("Attention {} is not implemented".format(name))

    def forward(self, x):
        return self.attention(x)

class DecoderBlock2d(nn.Module):
    def __init__(
        self,
        in_channels,
        skip_channels,
        out_channels,
        norm_layer: nn.Module = nn.Identity,
        attention_type: str = None,
        intermediate_conv: bool = False,
        upsample_mode: str = "deconv",
        scale_factor: int = 2,
    ):
        super().__init__()

        # Upsample block
        if upsample_mode == "pixelshuffle":
            self.upsample= SubpixelUpsample(
                spatial_dims= 2,
                in_channels= in_channels,
                scale_factor= scale_factor,
            )
        else:
            self.upsample = UpSample(
                spatial_dims= 2,
                in_channels= in_channels,
                out_channels= in_channels,
                scale_factor= scale_factor,
                mode= upsample_mode,
            )

        if intermediate_conv:
            k= 3
            c= skip_channels if skip_channels != 0 else in_channels
            self.intermediate_conv = nn.Sequential(
                ConvBnAct2d(c, c, k, k//2),
                ConvBnAct2d(c, c, k, k//2),
                )
        else:
            self.intermediate_conv= None

        self.attention1 = Attention2d(
            name= attention_type, 
            in_channels= in_channels + skip_channels,
            )

        self.conv1 = ConvBnAct2d(
            in_channels + skip_channels,
            out_channels,
            kernel_size= 3,
            padding= 1,
            norm_layer= norm_layer,
        )

        self.conv2 = ConvBnAct2d(
            out_channels,
            out_channels,
            kernel_size= 3,
            padding= 1,
            norm_layer= norm_layer,
        )
        self.attention2 = Attention2d(
            name= attention_type, 
            in_channels= out_channels,
            )

    def forward(self, x, skip=None):
        x = self.upsample(x)

        if self.intermediate_conv is not None:
            if skip is not None:
                skip = self.intermediate_conv(skip)
            else:
                x = self.intermediate_conv(x)

        if skip is not None:
            x = torch.cat([x, skip], dim=1)
            x = self.attention1(x)

        x = self.conv1(x)
        x = self.conv2(x)
        x = self.attention2(x)
        return x


class UnetDecoder2d(nn.Module):
    def __init__(
        self,
        encoder_channels: tuple[int],
        skip_channels: tuple[int] = None,
        decoder_channels: tuple = (256, 128, 64, 32),
        scale_factors: tuple = (2,2,2,2),
        norm_layer: nn.Module = nn.Identity,
        attention_type: str = None,
        intermediate_conv: bool = False,
        upsample_mode: str = "deconv",
    ):
        super().__init__()
        
        if len(encoder_channels) == 4:
            decoder_channels= decoder_channels[1:]
        self.decoder_channels= decoder_channels
        
        if skip_channels is None:
            skip_channels= list(encoder_channels[1:]) + [0]

        # Build decoder blocks
        in_channels= [encoder_channels[0]] + list(decoder_channels[:-1])
        self.blocks = nn.ModuleList()

        for i, (ic, sc, dc) in enumerate(zip(in_channels, skip_channels, decoder_channels)):
            self.blocks.append(
                DecoderBlock2d(
                    ic, sc, dc, 
                    norm_layer= norm_layer,
                    attention_type= attention_type,
                    intermediate_conv= intermediate_conv,
                    upsample_mode= upsample_mode,
                    scale_factor= scale_factors[i],
                    )
            )

    def forward(self, feats: list[torch.Tensor]):
        res= [feats[0]]
        feats= feats[1:]

        # Decoder blocks
        for i, b in enumerate(self.blocks):
            skip= feats[i] if i < len(feats) else None
            res.append(
                b(res[-1], skip=skip),
                )
            
        return res

class SegmentationHead2d(nn.Module):
    def __init__(
        self,
        in_channels,
        out_channels,
        scale_factor: tuple[int] = (2,2),
        kernel_size: int = 3,
        mode: str = "nontrainable",
    ):
        super().__init__()
        self.conv= nn.Conv2d(
            in_channels, out_channels, kernel_size= kernel_size,
            padding= kernel_size//2
        )
        self.upsample = UpSample(
            spatial_dims= 2,
            in_channels= out_channels,
            out_channels= out_channels,
            scale_factor= scale_factor,
            mode= mode,
        )

    def forward(self, x):
        x = self.conv(x)
        x = self.upsample(x)
        return x

#############
## Encoder ##
#############

def _convnext_block_forward(self, x):
    shortcut = x
    x = self.conv_dw(x)

    if self.use_conv_mlp:
        x = self.norm(x)
        x = self.mlp(x)
    else:
        x = self.norm(x)
        x = x.permute(0, 2, 3, 1)
        x = x.contiguous()
        x = self.mlp(x)
        x = x.permute(0, 3, 1, 2)
        x = x.contiguous()

    if self.gamma is not None:
        x = x * self.gamma.reshape(1, -1, 1, 1)

    x = self.drop_path(x) + self.shortcut(shortcut)
    return x


class Net(nn.Module):
    def __init__(
        self,
        backbone: str,
        pretrained: bool = True,
    ):
        super().__init__()
        
        # Encoder
        self.backbone= timm.create_model(
            backbone,
            in_chans= 5,
            pretrained= pretrained,
            features_only= True,
            drop_path_rate=0.0,
            )
        ecs= [_["num_chs"] for _ in self.backbone.feature_info][::-1]

        # Decoder
        self.decoder= UnetDecoder2d(
            encoder_channels= ecs,
        )

        self.seg_head= SegmentationHead2d(
            in_channels= self.decoder.decoder_channels[-1],
            out_channels= 1,
            scale_factor= 1,
        )
        
        self._update_stem(backbone)
        
        self.replace_activations(self.backbone, log=True)
        self.replace_norms(self.backbone, log=True)
        self.replace_forwards(self.backbone, log=True)

    def _update_stem(self, backbone):
        if backbone.startswith("convnext"):

            # Update stride
            self.backbone.stem_0.stride = (4, 1)
            self.backbone.stem_0.padding = (0, 2)

            # Duplicate stem layer (to downsample height)
            with torch.no_grad():
                w = self.backbone.stem_0.weight
                new_conv= nn.Conv2d(w.shape[0], w.shape[0], kernel_size=(4, 4), stride=(4, 1), padding=(0, 1))
                new_conv.weight.copy_(w.repeat(1, (128//w.shape[1])+1, 1, 1)[:, :new_conv.weight.shape[1], :, :])
                new_conv.bias.copy_(self.backbone.stem_0.bias)

            self.backbone.stem_0= nn.Sequential(
                nn.ReflectionPad2d((1,1,80,80)),
                self.backbone.stem_0,
                new_conv,
            )

        else:
            raise ValueError("Custom striding not implemented.")
        pass

    def replace_activations(self, module, log=False):
        if log:
            print(f"Replacing all activations with GELU...")
        
        # Apply activations
        for name, child in module.named_children():
            if isinstance(child, (
                nn.ReLU, nn.LeakyReLU, nn.Mish, nn.Sigmoid, 
                nn.Tanh, nn.Softmax, nn.Hardtanh, nn.ELU, 
                nn.SELU, nn.PReLU, nn.CELU, nn.GELU, nn.SiLU,
            )):
                setattr(module, name, nn.GELU())
            else:
                self.replace_activations(child)

    def replace_norms(self, mod, log=False):
        if log:
            print(f"Replacing all norms with InstanceNorm...")
            
        for name, c in mod.named_children():

            # Get feature size
            n_feats= None
            if isinstance(c, (nn.BatchNorm2d, nn.InstanceNorm2d)):
                n_feats= c.num_features
            elif isinstance(c, (nn.GroupNorm,)):
                n_feats= c.num_channels
            elif isinstance(c, (nn.LayerNorm,)):
                n_feats= c.normalized_shape[0]

            if n_feats is not None:
                new = nn.InstanceNorm2d(
                    n_feats,
                    affine=True,
                    )
                setattr(mod, name, new)
            else:
                self.replace_norms(c)

    def replace_forwards(self, mod, log=False):
        if log:
            print(f"Replacing forward functions...")
            
        for name, c in mod.named_children():
            if isinstance(c, ConvNeXtBlock):
                c.forward = MethodType(_convnext_block_forward, c)
            else:
                self.replace_forwards(c)

        
    def proc_flip(self, x_in):
        x_in= torch.flip(x_in, dims=[-3, -1])
        x= self.backbone(x_in)
        x= x[::-1]

        # Decoder
        x= self.decoder(x)
        x_seg= self.seg_head(x[-1])
        x_seg= x_seg[..., 1:-1, 1:-1]
        x_seg= torch.flip(x_seg, dims=[-1])
        x_seg= x_seg * 1500 + 3000
        return x_seg

    def forward(self, batch):
        x= batch

        # Encoder
        x_in = x
        x= self.backbone(x)
        x= x[::-1]

        # Decoder
        x= self.decoder(x)
        x_seg= self.seg_head(x[-1])
        x_seg= x_seg[..., 1:-1, 1:-1]
        x_seg= x_seg * 1500 + 3000
    
        if self.training:
            return x_seg
        else:
            p1 = self.proc_flip(x_in)
            x_seg = torch.mean(torch.stack([x_seg, p1]), dim=0)
            return x_seg

Overwriting _model.py


In [12]:
%%writefile _utils.py

import datetime

def format_time(elapsed):
    elapsed_rounded = int(round((elapsed)))
    return str(datetime.timedelta(seconds=elapsed_rounded))

Overwriting _utils.py


In [13]:
%%writefile _train.py

import os
import time 
import random
import numpy as np
from tqdm import tqdm

import torch
import torch.nn as nn
from torch.amp import autocast, GradScaler

import torch.distributed as dist
from torch.utils.data import DistributedSampler
from torch.nn.parallel import DistributedDataParallel

from _cfg import cfg
from _dataset import CustomDataset
from _model import ModelEMA, Net, L1_SSIM_Loss
from _utils import format_time

def set_seed(seed=1234):
    random.seed(seed)
    os.environ["PYTHONHASHSEED"] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = False
    torch.backends.cudnn.benchmark = True

def setup(rank, world_size):
    torch.cuda.set_device(rank)
    dist.init_process_group("nccl", rank=rank, world_size=world_size)
    return

def cleanup():
    dist.barrier()
    dist.destroy_process_group()
    return

def main(cfg):

    # ========== Datasets / Dataloaders ==========
    if cfg.local_rank == 0:
        print("="*50)
        print("🔥 ENHANCED TRAINING WITH L1+SSIM LOSS")
        print("="*50)
        print("Loading data..")
    train_ds = CustomDataset(cfg=cfg, mode="train")
    sampler= DistributedSampler(
        train_ds, 
        num_replicas=cfg.world_size, 
        rank=cfg.local_rank,
    )
    train_dl = torch.utils.data.DataLoader(
        train_ds, 
        sampler= sampler,
        batch_size= cfg.batch_size, 
        num_workers= 4,
    )
    
    valid_ds = CustomDataset(cfg=cfg, mode="valid")
    sampler= DistributedSampler(
        valid_ds, 
        num_replicas=cfg.world_size, 
        rank=cfg.local_rank,
    )
    valid_dl = torch.utils.data.DataLoader(
        valid_ds, 
        sampler= sampler,
        batch_size= cfg.batch_size_val, 
        num_workers= 4,
    )

    # ========== Model / Optim ==========
    model = Net(backbone=cfg.backbone)
    model= model.to(cfg.local_rank)
    if cfg.ema:
        if cfg.local_rank == 0:
            print("Initializing EMA model..")
        ema_model = ModelEMA(
            model, 
            decay=cfg.ema_decay, 
            device=cfg.local_rank,
        )
    else:
        ema_model = None
    model= DistributedDataParallel(
        model, 
        device_ids=[cfg.local_rank], 
        )
    
    # 🚀 ENHANCED LOSS FUNCTION
    criterion = L1_SSIM_Loss(alpha=0.85)
    
    # Better optimizer
    optimizer = torch.optim.AdamW(model.parameters(), lr=2e-4, weight_decay=1e-4)
    scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=cfg.epochs)
    
    scaler = GradScaler()

    if cfg.local_rank == 0:
        print(f"🎯 Loss Function: L1+SSIM (α={criterion.alpha})")
        print(f"📊 Optimizer: AdamW (lr=2e-4, wd=1e-4)")
        print(f"📈 Scheduler: CosineAnnealingLR")
        print(f"🔄 EMA Decay: {cfg.ema_decay}")
        print("="*50)

    # ========== Training ==========
    if cfg.local_rank == 0:
        print("🚀 Starting Enhanced Training...")
        print("="*50)
    
    best_loss= 1_000_000
    val_loss= 1_000_000

    for epoch in range(0, cfg.epochs+1):
        if epoch != 0:
            tstart= time.time()
            train_dl.sampler.set_epoch(epoch)
    
            # Train loop
            model.train()
            total_loss = []
            for i, (x, y) in enumerate(train_dl):
                x = x.to(cfg.local_rank)
                y = y.to(cfg.local_rank)
        
                with autocast(cfg.device.type):
                    logits = model(x)
                    
                loss = criterion(logits, y)
        
                scaler.scale(loss).backward()
                scaler.unscale_(optimizer)
        
                torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)
        
                scaler.step(optimizer)
                scaler.update()
                optimizer.zero_grad()
    
                total_loss.append(loss.item())
                
                if ema_model is not None:
                    ema_model.update(model)
                    
                if cfg.local_rank == 0 and (len(total_loss) >= cfg.logging_steps or i == 0):
                    train_loss = np.mean(total_loss)
                    total_loss = []
                    current_lr = optimizer.param_groups[0]['lr']
                    print("Epoch {:2d}: Loss={:.4f} | Val MAE={:.2f} | LR={:.2e} | Time={} | Step={:4d}/{}".format(
                        epoch, 
                        train_loss,
                        val_loss,
                        current_lr,
                        format_time(time.time() - tstart),
                        i+1, 
                        len(train_dl), 
                    ))
            
            scheduler.step()
    
        # ========== Validation ==========
        model.eval()
        val_logits = []
        val_targets = []
        with torch.no_grad():
            for x, y in tqdm(valid_dl, disable=cfg.local_rank != 0, desc=f"Epoch {epoch} Validation"):
                x = x.to(cfg.local_rank)
                y = y.to(cfg.local_rank)
    
                with autocast(cfg.device.type):
                    if ema_model is not None:
                        out = ema_model.module(x)
                    else:
                        out = model(x)

                val_logits.append(out.cpu())
                val_targets.append(y.cpu())

            val_logits= torch.cat(val_logits, dim=0)
            val_targets= torch.cat(val_targets, dim=0)
                
            # Use L1 for validation metric (competition metric)
            l1_criterion = nn.L1Loss()
            loss = l1_criterion(val_logits, val_targets).item()

        # Gather loss
        v = torch.tensor([loss], device=cfg.local_rank)
        torch.distributed.all_reduce(v, op=dist.ReduceOp.SUM)
        val_loss = (v[0] / cfg.world_size).item()
    
        # ========== Save Models ==========
        stop_train = torch.tensor([0], device=cfg.local_rank)
        if cfg.local_rank == 0:
            es= cfg.early_stopping
            if val_loss < best_loss:
                improvement = best_loss - val_loss
                print(f"🎉 NEW BEST: {best_loss:.3f} → {val_loss:.3f} (↓{improvement:.3f})")
                print("💾 Saved best model...")
                best_loss = val_loss
                if ema_model is not None:
                    torch.save(ema_model.module.state_dict(), f'enhanced_model_best_{cfg.seed}.pt')
                else:
                    torch.save(model.module.state_dict(), f'enhanced_model_best_{cfg.seed}.pt')
        
                es["streak"] = 0
            else:
                es["streak"] += 1
                if es["streak"] > es["patience"]:
                    print(f"⏹️ Early stopping after {es['streak']} epochs without improvement")
                    stop_train = torch.tensor([1], device=cfg.local_rank)
        
        # Exits training on all ranks
        dist.broadcast(stop_train, src=0)
        if stop_train.item() == 1:
            break

    # Final training summary
    if cfg.local_rank == 0:
        print("="*50)
        print("🏁 TRAINING COMPLETED!")
        print(f"🏆 Best Validation MAE: {best_loss:.3f}")
        print(f"📊 Total Epochs: {epoch}")
        print("="*50)

    return


if __name__ == "__main__":

   # GPU Specs
   rank = int(os.environ["RANK"])
   world_size = int(os.environ["WORLD_SIZE"])
   _, total = torch.cuda.mem_get_info(device=rank)

   # Init
   setup(rank, world_size)
   time.sleep(rank)
   print(f"Rank: {rank}, World size: {world_size}, GPU memory: {total / 1024**3:.2f}GB", flush=True)
   time.sleep(world_size - rank)

   # Seed
   set_seed(cfg.seed+rank)

   # Run
   cfg.local_rank= rank
   cfg.world_size= world_size
   main(cfg)
   cleanup()

Overwriting _train.py


In [ ]:
from _cfg import cfg

if RUN_TRAIN:
    print("🚀 Starting Enhanced Training...")
    print("="*60)
    print("📋 Training Configuration:")
    print(f"   • Epochs: {cfg.epochs}")
    print(f"   • Batch Size: {cfg.batch_size}")
    print(f"   • Loss: L1+SSIM (α=0.85)")
    print(f"   • Optimizer: AdamW")
    print(f"   • EMA: {cfg.ema}")
    print("="*60)
    
    !OMP_NUM_THREADS=1 torchrun --nproc_per_node=2 _train.py

🚀 Starting Enhanced Training...
📋 Training Configuration:
   • Epochs: 50
   • Batch Size: 16
   • Loss: L1+SSIM (α=0.85)
   • Optimizer: AdamW
   • EMA: True
<frozen importlib._bootstrap_external>:1241: FutureWarning: The cuda.cudart module is deprecated and will be removed in a future release, please switch to use the cuda.bindings.runtime module instead.
<frozen importlib._bootstrap_external>:1241: FutureWarning: The cuda.cudart module is deprecated and will be removed in a future release, please switch to use the cuda.bindings.runtime module instead.
2025-05-23 03:40:43.200099: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2025-05-23 03:40:43.200759: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1747971

In [ ]:
import glob
import torch
import torch.nn as nn
import torch.nn.functional as F
from _cfg import cfg
from _model import Net, EnsembleModel

print("="*60)
print("📥 LOADING TRAINED MODELS")
print("="*60)

if RUN_VALID or RUN_TEST:
    models = []
    
    # Try to load our newly trained model first
    trained_models = glob.glob("enhanced_model_best_*.pt")
    if trained_models:
        print("🎯 Loading our enhanced trained model:")
        for f in sorted(trained_models):
            print(f"   Loading: {f}")
            m = Net(
                backbone="convnext_small.fb_in22k_ft_in1k",
                pretrained=False,
            )
            state_dict = torch.load(f, map_location=cfg.device, weights_only=True)
            m.load_state_dict(state_dict)
            models.append(m)
    
    # Also load pretrained baseline models for comparison if available
    baseline_models = glob.glob("/kaggle/input/simple-further-finetuned-bartley-open-models/*.pth")
    if baseline_models and len(models) == 0:
        print("📁 Loading baseline pretrained models:")
        for f in sorted(baseline_models):
            print(f"   Loading: {f}")
            m = Net(
                backbone="convnext_small.fb_in22k_ft_in1k",
                pretrained=False,
            )
            state_dict = torch.load(f, map_location=cfg.device, weights_only=True)
            state_dict = {k.removeprefix("_orig_mod."):v for k,v in state_dict.items()}
            m.load_state_dict(state_dict)
            models.append(m)
    
    if not models:
        print("❌ No models found! Train a model first.")
        RUN_VALID = False
        RUN_TEST = False
    else:
        # Create ensemble
        model = EnsembleModel(models)
        model = model.to(cfg.device)
        model = model.eval()
        print(f"✅ Loaded {len(models)} model(s) for ensemble")
        print("="*60)

In [ ]:
if RUN_VALID:
    print("📊 VALIDATION ANALYSIS")
    print("="*60)
    
    from tqdm import tqdm
    import numpy as np
    from _dataset import CustomDataset

    # Dataset / Dataloader
    valid_ds = CustomDataset(cfg=cfg, mode="valid")
    sampler = torch.utils.data.SequentialSampler(valid_ds)
    valid_dl = torch.utils.data.DataLoader(
        valid_ds, 
        sampler=sampler,
        batch_size=cfg.batch_size_val, 
        num_workers=4,
    )

    # Validation loop
    criterion = nn.L1Loss()
    val_logits = []
    val_targets = []
    
    print("🔄 Running validation...")
    with torch.no_grad():
        for x, y in tqdm(valid_dl, desc="Validating"):
            x = x.to(cfg.device)
            y = y.to(cfg.device)
    
            with torch.autocast(cfg.device.type):
                out = model(x)
    
            val_logits.append(out.cpu())
            val_targets.append(y.cpu())
    
        val_logits = torch.cat(val_logits, dim=0)
        val_targets = torch.cat(val_targets, dim=0)
    
        total_loss = criterion(val_logits, val_targets).item()
    
    # Dataset-wise scores
    ds_idxs = np.array([valid_ds.records])
    ds_idxs = np.repeat(ds_idxs, repeats=500)
    
    print("="*60)
    print("📈 VALIDATION RESULTS")
    print("="*60)
    print("Dataset-wise MAE Scores:")
    print("-" * 30)
    
    dataset_scores = {}
    with torch.no_grad():    
        for idx in sorted(np.unique(ds_idxs)):
            mask = ds_idxs == idx
            logits_ds = val_logits[mask]
            targets_ds = val_targets[mask]
    
            loss = criterion(val_logits[mask], val_targets[mask]).item()
            dataset_scores[idx] = loss
            print(f"{idx:15} {loss:6.2f}")
    
    print("-" * 30)
    print(f"🎯 OVERALL VAL MAE: {total_loss:.3f}")
    print("="*60)
    
    # Comparison with baseline (if we know baseline scores)
    baseline_scores = {
        'CurveFault_A': 4.57,
        'CurveFault_B': 88.19,
        'CurveVel_A': 12.23,
        'CurveVel_B': 46.92,
        'FlatFault_A': 2.99,
        'FlatFault_B': 33.53,
        'FlatVel_A': 1.62,
        'FlatVel_B': 8.71,
        'Style_A': 35.01,
        'Style_B': 55.71
    }
    baseline_overall = 28.95
    
    print("📊 COMPARISON WITH BASELINE")
    print("="*60)
    print("Dataset               Enhanced    Baseline    Improvement")
    print("-" * 55)
    
    improvements = []
    for dataset in sorted(dataset_scores.keys()):
        if dataset in baseline_scores:
            enhanced = dataset_scores[dataset]
            baseline = baseline_scores[dataset]
            improvement = baseline - enhanced
            improvements.append(improvement)
            status = "✅" if improvement > 0 else "❌"
            print(f"{dataset:15} {enhanced:8.2f} {baseline:8.2f} {improvement:8.2f} {status}")
    
    overall_improvement = baseline_overall - total_loss
    print("-" * 55)
    print(f"{'OVERALL':15} {total_loss:8.2f} {baseline_overall:8.2f} {overall_improvement:8.2f} {'✅' if overall_improvement > 0 else '❌'}")
    
    if overall_improvement > 0:
        improvement_pct = (overall_improvement / baseline_overall) * 100
        print(f"🎉 IMPROVEMENT: {improvement_pct:.1f}% better than baseline!")
    else:
        degradation_pct = abs(overall_improvement / baseline_overall) * 100
        print(f"⚠️  DEGRADATION: {degradation_pct:.1f}% worse than baseline")
    
    print("="*60)

In [ ]:
if RUN_TEST:
    print("🔮 GENERATING TEST PREDICTIONS")
    print("="*60)
    
    import csv
    import time
    import pandas as pd
    from _utils import format_time

    class TestDataset(torch.utils.data.Dataset):
        def __init__(self, test_files):
            self.test_files = test_files

        def __len__(self):
            return len(self.test_files)

        def __getitem__(self, i):
            test_file = self.test_files[i]
            test_stem = test_file.split("/")[-1].split(".")[0]
            return np.load(test_file), test_stem

    ss = pd.read_csv("/kaggle/input/waveform-inversion/sample_submission.csv")    
    row_count = 0
    t0 = time.time()
    
    test_files = sorted(glob.glob("/kaggle/input/open-wfi-test/test/*.npy"))
    x_cols = [f"x_{i}" for i in range(1, 70, 2)]
    fieldnames = ["oid_ypos"] + x_cols
    
    test_ds = TestDataset(test_files)
    test_dl = torch.utils.data.DataLoader(
        test_ds, 
        sampler=torch.utils.data.SequentialSampler(test_ds),
        batch_size=cfg.batch_size_val, 
        num_workers=4,
    )
    
    print(f"📁 Found {len(test_files)} test files")
    print("🔄 Generating predictions...")
    
    with open("enhanced_submission.csv", "wt", newline="") as csvfile:
        writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
        writer.writeheader()

        with torch.inference_mode():
            with torch.autocast(cfg.device.type):
                for inputs, oids_test in tqdm(test_dl, desc="Predicting"):
                    inputs = inputs.to(cfg.device)
            
                    outputs = model(inputs)
                            
                    y_preds = outputs[:, 0].cpu().numpy()
                    
                    for y_pred, oid_test in zip(y_preds, oids_test):
                        for y_pos in range(70):
                            row = dict(zip(x_cols, [y_pred[y_pos, x_pos] for x_pos in range(1, 70, 2)]))
                            row["oid_ypos"] = f"{oid_test}_y_{y_pos}"
            
                            writer.writerow(row)
                            row_count += 1

                            if row_count % 100_000 == 0:
                                csvfile.flush()
    
    t1 = format_time(time.time() - t0)
    print(f"⏱️  Inference Time: {t1}")
    print(f"📝 Generated {row_count:,} predictions")
    print("💾 Saved to: enhanced_submission.csv")
    print("="*60)

In [ ]:
if RUN_TEST:
    print("🎨 PREDICTION VISUALIZATION")
    print("="*60)
    
    import matplotlib.pyplot as plt 

    # Plot sample predictions
    fig, axes = plt.subplots(3, 5, figsize=(15, 9))
    axes = axes.flatten()

    n = min(len(outputs), len(axes))
    
    for i in range(n):
        img = outputs[i, 0].cpu().numpy()
        idx = oids_test[i]
    
        axes[i].imshow(img, cmap='seismic', aspect='auto')
        axes[i].set_title(f'{idx}', fontsize=10)
        axes[i].axis('off')

    for i in range(n, len(axes)):
        axes[i].axis('off')
    
    plt.suptitle('Enhanced Model Predictions - Velocity Models', fontsize=16, y=0.98)
    plt.tight_layout()
    plt.show()

print("="*60)
print("🎯 ENHANCED TRAINING COMPLETE!")
print("="*60)
print("Key Improvements:")
print("• ✅ L1+SSIM Loss Function")
print("• ✅ AdamW Optimizer with Weight Decay")
print("• ✅ Cosine Annealing LR Schedule")
print("• ✅ Enhanced EMA (decay=0.999)")
print("• ✅ Better Gradient Clipping")
print("• ✅ Detailed Progress Tracking")
if 'total_loss' in locals():
    print(f"• 🎯 Final Validation MAE: {total_loss:.3f}")
    if 'baseline_overall' in locals():
        improvement = baseline_overall - total_loss
        if improvement > 0:
            print(f"• 🚀 Improvement over baseline: {improvement:.3f} ({(improvement/baseline_overall)*100:.1f}%)")
print("="*60)